In [4]:
# coding:utf-8
 
import sys
import gensim
import numpy as np
 
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
from sklearn.cluster import KMeans
 
TaggededDocument = gensim.models.doc2vec.TaggedDocument
def get_datasest():
    with open("D:\python\src\TCM clustering\cut.txt", 'r',encoding="utf8") as f:
        docs = f.readlines()
    train_input = []
    for patent_id, patent_abstract_text in enumerate(docs):
        word_list = patent_abstract_text.split(' ')
        l = len(word_list)
        document = TaggededDocument(word_list, tags=[patent_id])
        train_input.append(document)
    return train_input
 
def train(train_input):
    model_doc2vec = Doc2Vec(train_input,min_count=1, window = 3, vector_size = 200, 
                       sample=1e-3, negative=5, workers=5)
    model_doc2vec.train(train_input, total_examples=model_doc2vec.corpus_count, epochs=5)
    model_doc2vec.save('D:\python\src\TCM clustering\model_doc2vec')
    return model_doc2vec
 
def cluster(train_input):
    infered_vectors_list = []
    print("loading doc2vec model...")
    model_doc2vec= Doc2Vec.load("D:\python\src\TCM clustering\model_doc2vec")
    i = 0
    for text, label in train_input:
        vector = model_doc2vec.infer_vector(text)
        infered_vectors_list.append(vector)
        i += 1
    print("training kmeans model...")
    kmean_model = KMeans(n_clusters=15)
    kmean_model.fit(infered_vectors_list)
    labels= kmean_model.predict(infered_vectors_list[0:7000])
   
    
    
    
    for i in range(15): #for each cluster
        f=open("D:\python\src\TCM clustering\classify_"+str(i)+".txt", 'w',
               encoding="utf8") 
        cluster_words=[]#word sequence of each cluster
        cluster_str=''# word sequence string of each cluster
        for j in range(0,7000):
            if(labels[j]==i): 
                text = train_input[j][0]  
                cluster_words=cluster_words+text
        cluster_str=' '.join(map(str,cluster_words))
        f.write(cluster_str)
        f.close()
    
        

    with open("D:\python\src\TCM clustering\classify.txt", 'w',encoding="utf8") as wf:
        for i in range(7000):
            string = ""
            text = train_input[i][0]
            for word in text:
                string = string + word+' '
            string = string + '\t'
            string = string + str(labels[i])
            string = string + '\n'
            wf.write(string)
        return labels


    
    
    
   
#     return cluster_centers



 
if __name__ == '__main__':
    train_input = get_datasest()
    model_doc2vec = train(train_input)
    cluster_centers = cluster(train_input)

d:\Users\think\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


loading doc2vec model...
training kmeans model...
